In [3]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import warnings
warnings.filterwarnings('ignore')


In [4]:
IMAGE_SIZE = [224 , 224]

In [5]:
train_path = 'train'
test_path = 'test'


In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3] , weights = 'imagenet' , include_top = False)

In [7]:
folder = 'train'
print(len(folder))

5


In [8]:
for layers in vgg.layers:
    layers.trainable=False

In [9]:
x = Flatten()(vgg.output)
prediction = Dense(2,activation = 'softmax')(x)
model = Model(inputs=vgg.input , outputs = prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01,decay=1e-5,momentum=0.9)
model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [11]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_set = datagen.flow_from_directory('train',batch_size=100,target_size=(224,224))
test_set = datagen.flow_from_directory('test',batch_size=100,target_size=(224,224))

Found 27 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [12]:
# test_dagagen = ImageDataGenerator(
#     preprocessing_function = preprocess_input,
#     rotation_range = 40,
#     width_shift_range = 0.2,
#     height_shift_range = 0.2,
#     shear_range = 0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

In [99]:
#!pip install datetime
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint , LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
checkpoint = ModelCheckpoint(filepath='mymodel.h5',verbose=1,save_best_only=True)
callbacks = [checkpoint]
start = datetime.now()
model.fit_generator(
    train_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=5,
    validation_steps=32,
    callbacks=callbacks,verbose=1)
duration = datetime.now() - start
print('training completed: ',duration)


Epoch 1/10
5/5 [==============================] - 406s 81s/step - loss: 5.8319 - accuracy: 0.5259 - val_loss: 7.6895 - val_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 7.68955, saving model to mymodel.h5
Epoch 2/10
5/5 [==============================] - 282s 56s/step - loss: 7.4047 - accuracy: 0.5185 - val_loss: 7.6895 - val_accuracy: 0.5000

Epoch 00002: val_loss did not improve from 7.68955
Epoch 3/10
5/5 [==============================] - 221s 44s/step - loss: 7.4047 - accuracy: 0.5185 - val_loss: 7.6895 - val_accuracy: 0.5000

Epoch 00003: val_loss did not improve from 7.68955
Epoch 4/10
5/5 [==============================] - 213s 43s/step - loss: 7.4047 - accuracy: 0.5185 - val_loss: 7.6895 - val_accuracy: 0.5000

Epoch 00004: val_loss did not improve from 7.68955
Epoch 5/10
5/5 [==============================] - 210s 42s/step - loss: 7.4047 - accuracy: 0.5185 - val_loss: 7.6895 - val_accuracy: 0.5000

Epoch 00005: val_loss did not improve from 7.68955
Epoch 6/10
5

In [13]:
from keras.models import load_model
import cv2
img = cv2.imread('test/lepord/leopard.jpg')
model = load_model('mymodel.h5')
dim = (224,224)
img = cv2.resize(img , dim , interpolation=cv2.INTER_AREA)
x = image.img_to_array(img)
x = np.expand_dims(x , axis=0)
x = preprocess_input(x)
preds = model.predict(x)

In [15]:
preds

array([[0., 1.]], dtype=float32)